# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# data

In [2]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('LocalDisease/X_1_1.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('LocalDisease/Y_data_X_1_1_pca.csv', header=None, delimiter=',').values


train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_21, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)



torch.set_default_dtype(torch.float32)

# Emulator

In [3]:
Device = 'cpu'

In [4]:
checkpoint = torch.load('multitask_gp_checkpoint_21.pth', map_location=Device)
model_params = checkpoint['model_params']

MVGP_models = GP_models.MultitaskVariationalGP(train_x, train_y_21, 
                                               num_latents=model_params['num_latents'], 
                                               num_inducing=model_params['num_inducing'], 
                                               covar_type=model_params['covar_type']).to(Device)

MVGP_models.load_state_dict(checkpoint['model_state_dict'])

MVGP_likelihoods = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y_21.shape[1]).to(Device)
MVGP_likelihoods.load_state_dict(checkpoint['likelihood_state_dict'])

MVGP_models.eval()
MVGP_likelihoods.eval()


C:\Users\r4718\AppData\Local\Temp\ipykernel_27356\3521656856.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('multitask_gp_checkpoint_21.pth', ma

MultitaskGaussianLikelihood(
  (raw_task_noises_constraint): GreaterThan(1.000E-04)
  (raw_noise_constraint): GreaterThan(1.000E-04)
)

# PE

In [15]:
row_idx = 0

input_point = test_y_21[row_idx, :]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_21, k=5)

In [ ]:
local_train_y

In [16]:
local_train_x

tensor([[4.6552, 3.5896, 3.9173, 4.7579, 4.3711, 2.4636, 4.5686, 2.2586, 4.6105,
         4.0197],
        [3.7693, 4.0131, 4.8567, 4.8264, 4.4863, 3.9119, 4.7349, 3.7537, 3.2805,
         3.3339],
        [3.8011, 2.0099, 4.3266, 4.3820, 4.8882, 3.7963, 3.2047, 3.3966, 4.8571,
         4.6984],
        [3.6606, 3.2776, 3.7162, 2.7483, 2.8697, 4.6949, 4.8650, 4.9870, 4.0671,
         4.4954],
        [3.5507, 2.8556, 4.4150, 2.1824, 4.3511, 3.2741, 4.1643, 4.8750, 3.6670,
         3.7184]])

In [6]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)
    ).cpu().detach().numpy()

In [9]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x.to(Device)
    ).cpu().detach().numpy()

In [13]:
preds_tmp.view(100,21)

ValueError: Type must be a sub-type of ndarray type

In [12]:
test_y_21

tensor([[ 5.6642, 19.1396,  9.0095,  ...,  0.7106,  0.4195, -0.0979],
        [ 5.8902, 19.0561,  9.0718,  ...,  0.8444,  0.7396,  0.1433],
        [ 4.9911, 18.2672,  7.9840,  ...,  0.7221,  0.3132, -0.1671],
        ...,
        [ 5.2907, 17.5719,  9.9906,  ...,  0.8849,  0.3002, -0.1279],
        [ 6.1665, 20.3814,  8.2374,  ...,  0.4276,  0.7033,  0.0570],
        [ 5.0762, 16.5272, 10.5249,  ...,  0.8910,  0.3158, -0.1071]])

In [8]:
input_point

tensor([ 5.6642, 19.1396,  9.0095,  2.4783,  0.8886,  2.9374, -0.4040,  0.4124,
         1.4459,  0.6705,  1.2274, -2.2649, -0.0786, -1.8597, -0.7425, -0.6046,
        -1.1419,  2.1348,  0.7106,  0.4195, -0.0979])

In [ ]:


bounds = bound.get_bounds(local_train_x)

estimated_params_tmp, _ = Estimation.multi_start_estimation(
    MVGP_models, MVGP_likelihoods, row_idx, test_y_21, bounds,
    Estimation.estimate_params_for_one_model_Adam, num_starts=8, num_iterations=2000, lr=0.01,
    patience=10, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
)